In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
from scipy import stats
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
warnings.filterwarnings('ignore')

In [3]:
# set style for visualizations
plt.style.use('default')

sns.set_palette("husl")

In [6]:
print("Sales Performance EDA Project")
print("="*50)

Sales Performance EDA Project


## DATA GENERATION AND PREPARATION

In [4]:
def generate_sales_data(n_records=10000):
    """Generate realistic sales data for analysis"""
    
    np.random.seed(42)

In [5]:
# Dara range: 2 years of data
start_date = datetime(2022, 1, 1)
end_date = datetime(2023, 12, 31)

In [6]:
import pandas as pd
import numpy as np
from datetime import datetime

def generate_sales_data(n_records):
    """
    Generate synthetic sales data with realistic patterns and seasonality
    """
    # Generate dates with seasonality
    dates = []
    for _ in range(n_records):
        # Add seasonality - more sales in Nov-Dec and summer months
        month_weights = [0.8, 0.7, 0.9, 1.0, 1.1, 1.2, 1.1, 1.0, 0.9, 1.0, 1.3, 1.4]
        month = np.random.choice(range(1, 13), p=np.array(month_weights)/sum(month_weights))
        
        if month in [11, 12]:  # Holiday season
            day = np.random.randint(1, 29)
        else:
            day = np.random.randint(1, 29)
            
        year = np.random.choice([2022, 2023])
        dates.append(datetime(year, month, day))
    
    # Product categories and their characteristics
    products = {
        'Electronics': {'price_range': (50, 2000), 'margin': 0.25, 'seasonal_boost': 1.2},
        'Clothing': {'price_range': (20, 300), 'margin': 0.45, 'seasonal_boost': 1.1},
        'Home & Garden': {'price_range': (15, 500), 'margin': 0.35, 'seasonal_boost': 1.0},
        'Books': {'price_range': (10, 80), 'margin': 0.40, 'seasonal_boost': 0.9},
        'Sports': {'price_range': (25, 800), 'margin': 0.30, 'seasonal_boost': 1.15}
    }
    
    # Customer segments
    customer_segments = {
        'Premium': {'avg_order': 250, 'frequency': 'high', 'price_sensitivity': 0.2},
        'Regular': {'avg_order': 120, 'frequency': 'medium', 'price_sensitivity': 0.5},
        'Budget': {'avg_order': 60, 'frequency': 'low', 'price_sensitivity': 0.8}
    }
    
    # Generate data
    data = []
    customer_id = 1
    
    for i in range(n_records):
        # Customer segment
        segment = np.random.choice(list(customer_segments.keys()), 
                                 p=[0.2, 0.5, 0.3])
        
        # Product category
        category = np.random.choice(list(products.keys()))
        
        # Base price
        price_min, price_max = products[category]['price_range']
        base_price = np.random.uniform(price_min, price_max)
        
        # Apply seasonal adjustment
        month = dates[i].month
        if month in [11, 12]:  # Holiday season
            seasonal_factor = products[category]['seasonal_boost']
        elif month in [6, 7, 8]:  # Summer
            seasonal_factor = 1.1 if category in ['Sports', 'Clothing'] else 1.0
        else:
            seasonal_factor = 1.0
            
        # Customer segment price adjustment
        segment_factor = {
            'Premium': np.random.uniform(1.2, 1.8),
            'Regular': np.random.uniform(0.8, 1.2),
            'Budget': np.random.uniform(0.5, 0.9)
        }[segment]
        
        unit_price = base_price * seasonal_factor * segment_factor
        
        # Quantity (influenced by segment and price)
        if segment == 'Premium':
            quantity = np.random.poisson(2) + 1
        elif segment == 'Regular':
            quantity = np.random.poisson(1.5) + 1
        else:
            quantity = np.random.poisson(1) + 1
            
        # Revenue calculation
        revenue = unit_price * quantity
        
        # Cost and profit
        cost_per_unit = unit_price * (1 - products[category]['margin'])
        total_cost = cost_per_unit * quantity
        profit = revenue - total_cost
        
        # Geographic region
        region = np.random.choice(['North', 'South', 'East', 'West'], 
                                p=[0.3, 0.25, 0.25, 0.2])
        
        # Sales channel
        channel = np.random.choice(['Online', 'Store', 'Mobile'], 
                                 p=[0.5, 0.35, 0.15])
        
        data.append({
            'transaction_id': f'TXN_{i+1:06d}',
            'date': dates[i],
            'customer_id': f'CUST_{customer_id:05d}',
            'customer_segment': segment,
            'product_category': category,
            'unit_price': round(unit_price, 2),
            'quantity': quantity,
            'revenue': round(revenue, 2),
            'cost': round(total_cost, 2),
            'profit': round(profit, 2),
            'region': region,
            'sales_channel': channel
        })
        
        # Increment customer ID occasionally to simulate repeat customers
        if np.random.random() > 0.7:
            customer_id += 1
    
    return pd.DataFrame(data)

# Generate the dataset
print("Generating sales data...")
df = generate_sales_data(15000)

# Add derived features
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['quarter'] = df['date'].dt.quarter
df['day_of_week'] = df['date'].dt.day_name()
df['profit_margin'] = (df['profit'] / df['revenue']) * 100

print(f"Dataset created with {len(df)} records")
print(f"Date range: {df['date'].min()} to {df['date'].max()}")
print(f"Unique customers: {df['customer_id'].nunique()}")

# Display basic info about the dataset
print("\nDataset Info:")
print(df.info())
print("\nFirst few rows:")
print(df.head())

Generating sales data...
Dataset created with 15000 records
Date range: 2022-01-01 00:00:00 to 2023-12-28 00:00:00
Unique customers: 4505

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   transaction_id    15000 non-null  object        
 1   date              15000 non-null  datetime64[ns]
 2   customer_id       15000 non-null  object        
 3   customer_segment  15000 non-null  object        
 4   product_category  15000 non-null  object        
 5   unit_price        15000 non-null  float64       
 6   quantity          15000 non-null  int64         
 7   revenue           15000 non-null  float64       
 8   cost              15000 non-null  float64       
 9   profit            15000 non-null  float64       
 10  region            15000 non-null  object        
 11  sales_channel     15000 non-nul

##  BASIC DATA EXPLORATION

In [10]:
print("\n" + "="*50)
print("BASIC DATA EXPLORATION")
print("="*50)


BASIC DATA EXPLORATION


In [11]:
print("\nDataset Info:")
print(df.info())


Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   transaction_id    15000 non-null  object        
 1   date              15000 non-null  datetime64[ns]
 2   customer_id       15000 non-null  object        
 3   customer_segment  15000 non-null  object        
 4   product_category  15000 non-null  object        
 5   unit_price        15000 non-null  float64       
 6   quantity          15000 non-null  int64         
 7   revenue           15000 non-null  float64       
 8   cost              15000 non-null  float64       
 9   profit            15000 non-null  float64       
 10  region            15000 non-null  object        
 11  sales_channel     15000 non-null  object        
 12  year              15000 non-null  int32         
 13  month             15000 non-null  int32         
 14  quarter

In [12]:
print("\nFirst 5 rows:")
print(df.head())


First 5 rows:
  transaction_id       date customer_id customer_segment product_category  \
0     TXN_000001 2023-08-24  CUST_00001           Budget            Books   
1     TXN_000002 2022-06-27  CUST_00002           Budget      Electronics   
2     TXN_000003 2022-02-06  CUST_00002          Regular    Home & Garden   
3     TXN_000004 2023-05-23  CUST_00002          Regular            Books   
4     TXN_000005 2023-07-26  CUST_00002          Regular         Clothing   

   unit_price  quantity  revenue    cost  profit region sales_channel  year  \
0       24.28         4    97.11   58.27   38.85  South        Online  2023   
1      902.26         1   902.26  676.70  225.57   West         Store  2022   
2      148.09         2   296.19  192.52  103.67  North        Mobile  2022   
3       32.17         3    96.51   57.91   38.61   West        Online  2023   
4      236.31         2   472.62  259.94  212.68   West        Online  2023   

   month  quarter day_of_week  profit_margin  


In [13]:
print("\nBasic Statistics:")
print(df.describe())


Basic Statistics:
                                date    unit_price      quantity  \
count                          15000  15000.000000  15000.000000   
mean   2023-01-15 11:40:07.679999744    395.855516      2.450600   
min              2022-01-01 00:00:00      5.100000      1.000000   
25%              2022-07-16 00:00:00     70.527500      1.000000   
50%              2023-01-05 00:00:00    211.750000      2.000000   
75%              2023-07-13 00:00:00    490.515000      3.000000   
max              2023-12-28 00:00:00   3965.860000      9.000000   
std                              NaN    506.637092      1.271404   

            revenue          cost        profit          year         month  \
count  15000.000000  15000.000000  15000.000000  15000.000000  15000.000000   
mean    1009.633737    712.604769    297.028997   2022.506333      6.987600   
min        5.350000      3.210000      2.140000   2022.000000      1.000000   
25%      150.852500     91.707500     56.912500   20

In [14]:
print("\nMissing Values:")
print(df.isnull().sum())


Missing Values:
transaction_id      0
date                0
customer_id         0
customer_segment    0
product_category    0
unit_price          0
quantity            0
revenue             0
cost                0
profit              0
region              0
sales_channel       0
year                0
month               0
quarter             0
day_of_week         0
profit_margin       0
dtype: int64


## REVENUE TRENDS AND SEASONALITY ANALYSIS

In [15]:
print("\n" + "="*50)
print("REVENUE TRENDS AND SEASONALITY ANALYSIS")
print("="*50)


REVENUE TRENDS AND SEASONALITY ANALYSIS


In [16]:
# Monthly revenue trends
monthly_revenue = df.groupby(['year', 'month']).agg({
    'revenue': 'sum',
    'transaction_id': 'count',
    'profit': 'sum'
}).reset_index()  # Fixed: was reset.index()

monthly_revenue['date'] = pd.to_datetime(monthly_revenue[['year', 'month']].assign(day=1))
monthly_revenue['avg_order_value'] = monthly_revenue['revenue'] / monthly_revenue['transaction_id']

print("\nMonthly Revenue Summary:")
print(monthly_revenue.groupby('month')['revenue'].describe())  # Fixed: was monthy_revenue

# Seasonality analysis
quarter_stats = df.groupby('quarter').agg({
    'revenue': ['sum', 'mean', 'std'],
    'transaction_id': 'count',  # Fixed: removed the colon
    'profit': 'sum'
}).round(2)

print("\nQuarterly Performance:")  # Fixed: added missing 'n'
print(quarter_stats)

# Day of week analysis
dow_analysis = df.groupby('day_of_week').agg({  # Fixed: was 'day of week'
    'revenue': ['sum', 'mean', 'count'],
    'profit': 'sum'
}).round(2)

print("\nDay of Week Analysis:")
print(dow_analysis)

# Statistical test for seasonality
from scipy.stats import f_oneway  # Fixed: was 'fron'

# Test if quarterly revenues are significantly different
q1_revenue = df[df['quarter'] == 1]['revenue']
q2_revenue = df[df['quarter'] == 2]['revenue']
q3_revenue = df[df['quarter'] == 3]['revenue']
q4_revenue = df[df['quarter'] == 4]['revenue']

f_stat, p_value = f_oneway(q1_revenue, q2_revenue, q3_revenue, q4_revenue)
print(f"\nANOVA Test for Quarterly Revenue Differences:")
print(f"F-statistic: {f_stat:.4f}")
print(f"P-value: {p_value:.4f}")
print(f"Seasonal effect significant: {'Yes' if p_value < 0.05 else 'No'}")


Monthly Revenue Summary:
       count         mean            std         min           25%  \
month                                                                
1        2.0   529420.350    6302.103329   524964.09  5.271922e+05   
2        2.0   413643.720     469.943167   413311.42  4.134776e+05   
3        2.0   541979.310   76066.955418   488191.85  5.150856e+05   
4        2.0   570880.110  139145.589631   472489.32  5.216847e+05   
5        2.0   619866.310   16615.962840   608117.05  6.139917e+05   
6        2.0   690066.765   76935.756306   635664.97  6.628659e+05   
7        2.0   698082.900   13362.564540   688634.14  6.933585e+05   
8        2.0   587233.465   35024.010035   562467.75  5.748506e+05   
9        2.0   456707.270   47681.978022   422991.02  4.398491e+05   
10       2.0   561089.910   32278.236622   538265.75  5.496778e+05   
11       2.0   871466.955   66687.233463   824311.96  8.478895e+05   
12       2.0  1031815.960   39108.336585  1004162.19  1.017989e+

## CUSTOMER SEGMENTATION AND BEHAVOUR ANALYSIS

In [17]:
print("\n" + "="*50)
print("CUSTOMER SEGMENTATION AND BEHAVIOR ANALYSIS")
print("="*50)

# Customer-level aggregations
customer_stats = df.groupby(['customer_id', 'customer_segment']).agg({
    'revenue': ['sum', 'mean', 'count'],
    'profit': 'sum',
    'quantity': 'sum',
    'date': ['min', 'max']
}).reset_index()

# Flatten column names
customer_stats.columns = ['customer_id', 'customer_segment', 'total_revenue', 
                         'avg_order_value', 'total_orders', 'total_profit', 
                         'total_quantity', 'first_purchase', 'last_purchase']

# Calculate customer lifetime value metrics
customer_stats['customer_lifetime_days'] = (customer_stats['last_purchase'] - 
                                           customer_stats['first_purchase']).dt.days + 1
customer_stats['avg_days_between_orders'] = (customer_stats['customer_lifetime_days'] / 
                                            customer_stats['total_orders'])

print("\nCustomer Segment Analysis:")
segment_analysis = customer_stats.groupby('customer_segment').agg({
    'total_revenue': ['mean', 'median', 'std'],
    'avg_order_value': ['mean', 'median'],
    'total_orders': ['mean', 'median'],
    'total_profit': 'mean',
    'customer_lifetime_days': 'mean'
}).round(2)

print(segment_analysis)

# RFM Analysis (Recency, Frequency, Monetary)
reference_date = df['date'].max()
rfm = df.groupby('customer_id').agg({
    'date': lambda x: (reference_date - x.max()).days,  # Recency
    'transaction_id': 'count',  # Frequency
    'revenue': 'sum'  # Monetary
}).reset_index()

rfm.columns = ['customer_id', 'recency', 'frequency', 'monetary']

# RFM Scoring
rfm['r_score'] = pd.qcut(rfm['recency'], 5, labels=[5,4,3,2,1])
rfm['f_score'] = pd.qcut(rfm['frequency'].rank(method='first'), 5, labels=[1,2,3,4,5])
rfm['m_score'] = pd.qcut(rfm['monetary'], 5, labels=[1,2,3,4,5])

rfm['rfm_score'] = rfm['r_score'].astype(str) + rfm['f_score'].astype(str) + rfm['m_score'].astype(str)

print(f"\nRFM Analysis Summary:")
print(f"Average Recency: {rfm['recency'].mean():.1f} days")
print(f"Average Frequency: {rfm['frequency'].mean():.1f} orders")
print(f"Average Monetary: ${rfm['monetary'].mean():.2f}")

# K-means clustering for customer segmentation
features_for_clustering = ['recency', 'frequency', 'monetary']
scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(rfm[features_for_clustering])

# Determine optimal number of clusters using elbow method
inertias = []
k_range = range(2, 8)
for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(rfm_scaled)
    inertias.append(kmeans.inertia_)

# Use 4 clusters
optimal_k = 4
kmeans = KMeans(n_clusters=optimal_k, random_state=42)
rfm['cluster'] = kmeans.fit_predict(rfm_scaled)

print(f"\nCustomer Clusters (K-means with k={optimal_k}):")
cluster_summary = rfm.groupby('cluster')[features_for_clustering].mean().round(2)
print(cluster_summary)


CUSTOMER SEGMENTATION AND BEHAVIOR ANALYSIS

Customer Segment Analysis:
                 total_revenue                   avg_order_value          \
                          mean   median      std            mean  median   
customer_segment                                                           
Budget                  943.77   528.19  1191.30          560.41  334.83   
Premium                2627.19  1369.68  3463.84         1797.02  958.29   
Regular                2095.19  1111.61  2540.25          969.19  629.03   

                 total_orders        total_profit customer_lifetime_days  
                         mean median         mean                   mean  
customer_segment                                                          
Budget                   1.71    1.0       277.33                 129.81  
Premium                  1.47    1.0       770.36                  92.26  
Regular                  2.14    2.0       618.11                 182.56  

RFM Analysis Summar

## PRODUCT PERFORMANCE ANALYSIS

In [19]:
print("\n" + "="*50)
print("PRODUCT PERFORMANCE ANALYSIS")
print("="*50)


PRODUCT PERFORMANCE ANALYSIS


In [26]:
print("\n" + "="*50)
print("PRODUCT PERFORMANCE ANALYSIS")
print("="*50)

# Product category performance
product_performance = df.groupby('product_category').agg({
    'revenue': ['sum', 'mean', 'count'],
    'profit': ['sum', 'mean'],
    'profit_margin': 'mean',
    'quantity': 'sum'
}).round(2)

print("Product Category Performance:")
print(product_performance)

# Channel performance by product
channel_product = df.groupby(['sales_channel', 'product_category']).agg({
    'revenue': 'sum',
    'profit': 'sum',
    'transaction_id': 'count'
}).reset_index()

print("\nChannel Performance by Product Category:")
channel_pivot = channel_product.pivot_table(
    values='revenue', 
    index='product_category', 
    columns='sales_channel', 
    fill_value=0
).round(2)
print(channel_pivot)

# Regional performance
regional_performance = df.groupby(['region', 'product_category']).agg({
    'revenue': 'sum',
    'profit': 'sum'
}).reset_index()

print("\nRegional Performance:")
regional_pivot = regional_performance.pivot_table(
    values='revenue',
    index='region',
    columns='product_category',
    fill_value=0
).round(2)
print(regional_pivot)

# Product profitability analysis
profitability = df.groupby('product_category').agg({
    'profit_margin': ['mean', 'std'],
    'revenue': 'sum',
    'profit': 'sum'
}).round(2)

print("\nProduct Profitability Analysis:")
print(profitability)


PRODUCT PERFORMANCE ANALYSIS
Product Category Performance:
                     revenue                     profit         profit_margin  \
                         sum     mean count         sum    mean          mean   
product_category                                                                
Books              349671.83   113.79  3073   139869.10   45.52          40.0   
Clothing          1265160.53   430.47  2939   569322.49  193.71          45.0   
Electronics       8203010.38  2774.10  2957  2050752.42  693.52          25.0   
Home & Garden     1949840.74   651.47  2993   682444.31  228.01          35.0   
Sports            3376822.57  1111.53  3038  1013046.63  333.46          30.0   

                 quantity  
                      sum  
product_category           
Books                7558  
Clothing             7212  
Electronics          7304  
Home & Garden        7319  
Sports               7366  

Channel Performance by Product Category:
sales_channel         Mob

## STATISTICAL TESTING AND CORRELATION ANALYSIS

In [30]:
print("\n" + "="*50)
print("STATISTICAL TESTING AND CORRELATION ANALYSIS")

# Correlation analysis
numeric_cols = ['unit_price', 'quantity', 'revenue', 'profit', 'profit_margin']
correlation_matrix = df[numeric_cols].corr()

print("Correlation Matrix:")
print(correlation_matrix.round(3))


# Statistical tests

# 1. Test if Premium customers have significantly higher order values
premium_orders = df[df['customer_segment'] == 'Premium']['revenue']
regular_orders = df[df['customer_segment'] == 'Regular']['revenue']
budget_orders = df[df['customer_segment'] == 'Budget']['revenue']

# T-test between Premium and Regular
t_stat, p_val = stats.ttest_ind(premium_orders, regular_orders)
print(f"\nT-test: Premium vs Regular customer order values")
print(f"T-statistic: {t_stat:.4f}, P-value: {p_val:.4f}")
print(f"Premium customers have significantly higher orders: {'Yes' if p_val < 0.05 else 'No'}")

# 2. ANOVA test for sales channels
online_revenue = df[df['sales_channel'] == 'Online']['revenue']
store_revenue = df[df['sales_channel'] == 'Store']['revenue']
mobile_revenue = df[df['sales_channel'] == 'Mobile']['revenue']

f_stat, p_val = f_oneway(online_revenue, store_revenue, mobile_revenue)
print(f"\nANOVA: Revenue differences across sales channels")
print(f"F-statistic: {f_stat:.4f}, P-value: {p_val:.4f}")
print(f"Significant channel differences: {'Yes' if p_val < 0.05 else 'No'}")

# 3. Chi-square test for independence (Region vs Product Category)
contingency_table = pd.crosstab(df['region'], df['product_category'])
chi2, p_val, dof, expected = stats.chi2_contingency(contingency_table)

print(f"\nChi-square test: Region vs Product Category independence")
print(f"Chi-square: {chi2:.4f}, P-value: {p_val:.4f}")
print(f"Region and Product Category are independent: {'No' if p_val < 0.05 else 'Yes'}")

# 4. Regression analysis - factors affecting profit margin
from scipy.stats import pearsonr

# Test correlations with profit margin
price_corr, price_p = pearsonr(df['unit_price'], df['profit_margin'])
quantity_corr, quantity_p = pearsonr(df['quantity'], df['profit_margin'])

print(f"\nCorrelation with Profit Margin:")
print(f"Unit Price: r={price_corr:.3f}, p={price_p:.4f}")
print(f"Quantity: r={quantity_corr:.3f}, p={quantity_p:.4f}")


STATISTICAL TESTING AND CORRELATION ANALYSIS
Correlation Matrix:
               unit_price  quantity  revenue  profit  profit_margin
unit_price          1.000     0.061    0.847   0.821         -0.616
quantity            0.061     1.000    0.375   0.428          0.000
revenue             0.847     0.375    1.000   0.990         -0.488
profit              0.821     0.428    0.990   1.000         -0.433
profit_margin      -0.616     0.000   -0.488  -0.433          1.000

T-test: Premium vs Regular customer order values
T-statistic: 20.3288, P-value: 0.0000
Premium customers have significantly higher orders: Yes

ANOVA: Revenue differences across sales channels
F-statistic: 0.0086, P-value: 0.9915
Significant channel differences: No

Chi-square test: Region vs Product Category independence
Chi-square: 11.6178, P-value: 0.4768
Region and Product Category are independent: Yes

Correlation with Profit Margin:
Unit Price: r=-0.616, p=0.0000
Quantity: r=0.000, p=0.9626


## ADVANCED ANALYTICS

In [31]:

print("\n" + "="*50)
print("ADVANCED ANALYTICS")
print("="*50)

# Time series decomposition (simplified trend analysis)
monthly_revenue_ts = monthly_revenue.set_index('date')['revenue']

# Calculate growth rates
monthly_revenue['mom_growth'] = monthly_revenue['revenue'].pct_change() * 100
monthly_revenue['yoy_growth'] = monthly_revenue['revenue'].pct_change(periods=12) * 100

print("Monthly Growth Rates:")
print(f"Average MoM Growth: {monthly_revenue['mom_growth'].mean():.2f}%")
print(f"Average YoY Growth: {monthly_revenue['yoy_growth'].mean():.2f}%")

# Customer lifetime value calculation
avg_order_value = df['revenue'].mean()
purchase_frequency = df.groupby('customer_id')['transaction_id'].count().mean()
customer_lifespan = customer_stats['customer_lifetime_days'].mean() / 365.25

clv = avg_order_value * purchase_frequency * customer_lifespan
print(f"\nCustomer Lifetime Value Estimation:")
print(f"Average Order Value: ${avg_order_value:.2f}")
print(f"Average Purchase Frequency: {purchase_frequency:.2f} orders/customer")
print(f"Average Customer Lifespan: {customer_lifespan:.2f} years")
print(f"Estimated CLV: ${clv:.2f}")

# Product affinity analysis (simplified market basket)
customer_products = df.groupby('customer_id')['product_category'].apply(set).reset_index()
customer_products['num_categories'] = customer_products['product_category'].apply(len)

cross_category_customers = customer_products[customer_products['num_categories'] > 1]
print(f"\nCross-category Analysis:")
print(f"Customers buying multiple categories: {len(cross_category_customers)}")
print(f"Percentage of multi-category customers: {len(cross_category_customers)/len(customer_products)*100:.1f}%")


ADVANCED ANALYTICS
Monthly Growth Rates:
Average MoM Growth: 6.43%
Average YoY Growth: 6.42%

Customer Lifetime Value Estimation:
Average Order Value: $1009.63
Average Purchase Frequency: 3.32 orders/customer
Average Customer Lifespan: 0.39 years
Estimated CLV: $1314.70

Cross-category Analysis:
Customers buying multiple categories: 2939
Percentage of multi-category customers: 65.1%


## KEY INSIGHTS AND RECOMMENDATIONS

In [32]:
print("\n" + "="*50)
print("KEY INSIGHTS AND RECOMMENDATIONS")
print("="*50)

print("📊 REVENUE INSIGHTS:")
total_revenue = df['revenue'].sum()
total_profit = df['profit'].sum()
overall_margin = (total_profit / total_revenue) * 100

print(f"• Total Revenue: ${total_revenue:,.2f}")
print(f"• Total Profit: ${total_profit:,.2f}")
print(f"• Overall Profit Margin: {overall_margin:.1f}%")

best_month = monthly_revenue.loc[monthly_revenue['revenue'].idxmax(), 'month']
worst_month = monthly_revenue.loc[monthly_revenue['revenue'].idxmin(), 'month']
print(f"• Best performing month: {best_month}")
print(f"• Lowest performing month: {worst_month}")

print(f"\n👥 CUSTOMER INSIGHTS:")
print(f"• Total unique customers: {df['customer_id'].nunique():,}")
print(f"• Most valuable segment: {customer_stats.groupby('customer_segment')['total_revenue'].mean().idxmax()}")
print(f"• Average customer lifetime: {customer_lifespan:.1f} years")

top_products = product_performance.iloc[:, 0].sort_values(ascending=False)
print(f"\n🛍️ PRODUCT INSIGHTS:")
print(f"• Top revenue category: {top_products.index[0]}")
print(f"• Most profitable category: {profitability.iloc[:, 2].idxmax()}")

top_channel = df.groupby('sales_channel')['revenue'].sum().idxmax()
print(f"\n📱 CHANNEL INSIGHTS:")
print(f"• Top performing channel: {top_channel}")

print(f"\n🎯 RECOMMENDATIONS:")
print("1. Focus marketing efforts on Q4 (highest revenue quarter)")
print("2. Develop retention strategies for Budget segment customers")
print("3. Investigate why certain product categories underperform in specific regions")
print("4. Optimize mobile channel performance (lowest revenue)")
print("5. Implement cross-selling strategies for single-category customers")

print("\n" + "="*50)
print("ANALYSIS COMPLETE!")
print("="*50)

# Save key results for visualization
results = {
    'monthly_revenue': monthly_revenue,
    'customer_stats': customer_stats,
    'rfm': rfm,
    'product_performance': product_performance,
    'correlation_matrix': correlation_matrix,
    'df': df
}

print(f"\nDataset shape: {df.shape}")
print("Ready for visualization and further analysis!")


KEY INSIGHTS AND RECOMMENDATIONS
📊 REVENUE INSIGHTS:
• Total Revenue: $15,144,506.05
• Total Profit: $4,455,434.95
• Overall Profit Margin: 29.4%
• Best performing month: 12
• Lowest performing month: 2

👥 CUSTOMER INSIGHTS:
• Total unique customers: 4,514
• Most valuable segment: Premium
• Average customer lifetime: 0.4 years

🛍️ PRODUCT INSIGHTS:
• Top revenue category: Electronics
• Most profitable category: Electronics

📱 CHANNEL INSIGHTS:
• Top performing channel: Online

🎯 RECOMMENDATIONS:
1. Focus marketing efforts on Q4 (highest revenue quarter)
2. Develop retention strategies for Budget segment customers
3. Investigate why certain product categories underperform in specific regions
4. Optimize mobile channel performance (lowest revenue)
5. Implement cross-selling strategies for single-category customers

ANALYSIS COMPLETE!

Dataset shape: (15000, 17)
Ready for visualization and further analysis!
